In [1]:
import numpy as np
import pandas as pd

from helper import *

Write some information about your experiment.

`filepath bottom` and `filepath_side` are the paths for raw DLC outputs of the videos from bottom-view and side-view, respectively.

`FPS0` and `FPS1` are the FPS of the videos from bottom-view and side-view, respectively.


`time0` and `time1` are the time from two videos corresponding to the same moment in the real world: e.g., the following example means that the moment of 67.956s in the bottom-view video and the moment of 66.666s in the side-view video coincide.

`bottom fz` is the `(4, 2)`-shaped numpy array where each row represents the x- and y- coordinates of the upper left, upper right, lower left, and lower right vertices of the food zone when viewed from bottom, respectively. (Note that lower y-coordinate means upper position in the computer vision.)

`side_fz` is the `(2,)`-shaped numpy array where each element represents the y-coordinates of the upper edge and lower edge of the food zone when viewed from side, respectively.

`ls` is the `(N,)`-shaped numpy array of integer datatype (`N`=number of photostimulations) where each element represents the frame at which each session of photostimulation started.

`laser_length` is the time (in second) of each session of photostimulation.

One example is given below:

In [2]:
filepath_bottom = 'Examples/210817_LepR71-1_ChR2_bottomDLC_resnet50_lickAug24shuffle1_500000.csv'
filepath_side = 'Examples/210817_LepR71-1_ChR2_sideDLC_resnet50_lickSep16shuffle1_500000.csv'
FPS0, FPS1 = 23.69, 23.97
time0, time1 = 67.956, 66.666
bottom_fz = np.array([[346, 830],
                    [587, 830],
                    [325, 920],
                    [641, 920]])
side_fz = np.array([251, 462])
ls = np.array([1592, 3288, 4977, 6636, 8134, 10479, 13094, 17135, 19031, 21102], dtype=int)
laser_length = 10

Then one can run the following cell. We get:
* `coords`: coordinates from `filepath_bottom` and `filepath_side` with synchronization
* `start`, `end`: start and end of 'proper' frames with higher likelihood than our criterion (in our case, 0.9)

In [3]:
df_bottom = pd.read_csv(filepath_bottom, index_col=0, header=[1, 2], skiprows=0)
df_side = pd.read_csv(filepath_side, index_col=0, header=[1, 2], skiprows=0)
coords_bottom, start_bottom, end_bottom = lininterpol(df_bottom, required_bps_bottom, 0.9, True)
coords_side, start_side, end_side = lininterpol(df_side, required_bps_side, 0.9, True)
coords, start, end = merge(coords_bottom, coords_side, start_bottom, start_side, time0, time1, FPS0, FPS1)

In [4]:
coords

tail_base                  hand_L                  hand_R              \
                x           y           x           y           x           y   
0      651.402100  165.112640  214.724304  468.519562  434.146606  639.619263   
1      600.925781  144.938217  213.259460  467.919220  421.489532  652.707275   
2      582.231689  138.735626  213.425003  468.784180  404.749451  687.261658   
3      552.210022  114.360687  215.029221  469.503296  379.950699  701.214325   
4      532.330688   91.658615  212.650681  470.604797  355.151947  715.166992   
...           ...         ...         ...         ...         ...         ...   
22423  451.769989  152.089127  587.329285  650.960510  703.839294  611.383789   
22424  451.496368  149.417969  570.324524  648.794189  694.539734  624.058044   
22425  453.165924  150.919769  563.447754  656.911987  694.806091  635.217957   
22426  454.626740  150.028000  551.977417  664.944580  695.679321  658.980652   
22427  462.263519  142.071869  519.142456  677.420166  684.302795  722.799255   

            paw_L                   paw_R                   snout              \
                x           y           x           y           x           y   
0      479.265320   11.773515  480.860321  435.469391  333.168579  798.902039   
1      467.899109   14.687907  486.622498  444.025299  322.408203  821.370239   
2      362.248403  124.393409  488.425903  446.937408  280.291168  829.261047   
3      256.597697  234.098911  489.577332  448.419922  261.473083  837.477539   
4      150.946991  343.804413  493.659424  450.821320  248.846405  816.157715   
...           ...         ...         ...         ...         ...         ...   
22423  426.278717  545.274231  792.634155  445.284637  644.078247  763.157104   
22424  425.174072  544.038696  795.063171  448.129883  637.775452  788.006531   
22425  425.004913  544.266968  791.771790  445.340027  626.075317  812.305054   
22426  425.329620  542.996460  791.527771  444.613037  615.598633  839.346252   
22427  426.026886  542.515991  791.421570  444.649475  554.357727  893.379456   

       snout_side              
                x           y  
0      519.886394  453.035461  
1      517.166407  453.205438  
2      514.446420  453.375415  
3      511.726432  453.545392  
4      509.006445  453.715369  
...           ...         ...  
22423  547.286199  531.796153  
22424  566.181944  537.216547  
22425  585.077689  542.636940  
22426  593.973977  548.748677  
22427  595.145177  553.189371  

[22428 rows x 14 columns]

In [5]:
start, end

(68, 22495)

To determine whether each frame satisfies our four conditions, run the following cell: we get the `(4, len(coords))`-shaped numpy array `t` where each row represents whether each frame satisfies the corresponding condition. (To see our conditions, see `helper.py`.)

In [6]:
laser_frame_length = np.around(FPS0 * laser_length).astype(int)

t = np.vstack((conditions[0](coords), conditions[1](coords), conditions[2](coords, bottom_fz), conditions[3](coords, *side_fz)))

t

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [False, False, False, ..., False, False,  True],
       [ True,  True,  True, ..., False, False, False]])

Now we select the frames which will be further classified into 'Evaluating Behaviour (Lick)' and 'Swallowing Behaviour (Bite),' i.e., the frames satisfying all of our conditions, among the frames during photostimulation.

By running this cell, we get the dataframe `df` of length (in frames) of total photostimulation, i.e., `laser_frame_length * len(ls)`. `laseron` column contains all frames during the photostimulation sessions, and `near food` column shows whether each frame satisfies all of our conditions.

In [7]:
laser_frame_length = np.around(FPS0 * laser_length).astype(int)

t = np.vstack((conditions[0](coords), conditions[1](coords), conditions[2](coords, bottom_fz), conditions[3](coords, *side_fz)))
foi = t.all(axis=0)

ls_on = (ls.reshape(-1, 1) + np.arange(laser_frame_length).reshape(1, -1)).reshape(-1)
tmp = []
for j in ls_on:
    if j < start or j > end:
        tmp.append(False)
    else:
        tmp.append(foi[j - start])
df = pd.DataFrame(data={
    'laseron' : ls_on,
    'near food' : tmp
})

df

,laseron,near food
0,1592,False
1,1593,False
2,1594,False
3,1595,False
4,1596,False
...,...,...
2365,21334,True
2366,21335,True
2367,21336,True
2368,21337,True


To save `df`, write the path of csv file where `df` will be saved. Make sure that the folder containing this path already exists.

Running all of this example code produces the same file as `example_result.csv` in the `Examples` folder.

In [8]:
result_path = 'Examples/example_result.csv'
df.to_csv(result_path)